In [14]:
import pandas as pd
import numpy as np
import pymysql
import getpass

pd.set_option("display.max_rows", 500)


In [2]:
pw = getpass.getpass()

········


In [3]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="ironhack",
                       passwd=pw,
                       db="olist");

In [4]:
# 1. get tables
orders = pd.read_sql("SELECT * FROM orders;", conn)
order_items = pd.read_sql("SELECT * FROM order_items;", conn)
customers = pd.read_sql("SELECT * FROM customers;", conn)


In [8]:
# 2. join customers with orders to get the state for each order_id

orders_state = orders.merge(customers, on="customer_id")[["order_id", "order_purchase_timestamp", "customer_state"]]

In [20]:
# 3. join orders_state with order_items to get the price and sum by order_id

# (order_items
#  .merge(orders_state, on="order_id")
#  .groupby(["order_id","customer_state"])
#  .sum()[["order_id", "order_purchase_timestamp", "customer_state", "price"]]

# DIDNT WORK BECAUSE OF THE TIMESTAMP...

,,order_item_id,price,freight_value
order_id,customer_state,,,
00010242fe8c5a6d1ba2dd792cb16214,RJ,1,58.90,13.29
00018f77f2f0320c557190d7a144bdd3,SP,1,239.90,19.93
000229ec398224ef6ca0657da4fc703e,MG,1,199.00,17.87
00024acbcdf0a6daa1e931b038114c75,SP,1,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,SP,1,199.90,18.14
...,...,...,...,...
fffc94f6ce00a00581880bf54a75a037,MA,1,299.99,43.41
fffcd46ef2263f404302a634eb57f7eb,PR,1,350.00,36.53
fffce4705a9662cd70adb13d4a31832d,SP,1,99.90,16.95


In [26]:
# 4. Reduce order_items to order_id level first, then join

rev_per_order = (order_items
                 .groupby("order_id")
                 .sum()[["price"]]
                 .reset_index())

In [28]:
# 5. now join

order_state_rev = orders_state.merge(rev_per_order, on="order_id")

In [33]:
# 6. truncate to month

# Truncate timestamp to month:
# order_state_rev["order_purchase_timestamp"].dt.to_period("M")
order_state_rev = order_state_rev.assign(month_id=order_state_rev["order_purchase_timestamp"].dt.to_period("M"))

In [39]:
# 7. group by month and state
monthly_order_state_rev = order_state_rev.groupby(["month_id","customer_state"]).sum().reset_index()
monthly_order_state_rev = monthly_order_state_rev.rename({"price": "revenue"}, axis=1)

In [41]:
monthly_order_state_rev.head()

,month_id,customer_state,revenue
0,2016-09,RR,72.89
1,2016-09,RS,59.50
2,2016-09,SP,134.97
3,2016-10,AL,82.49
4,2016-10,BA,894.05


In [55]:
# 8. shift month_ids by 1 month
monthly_order_state_rev = monthly_order_state_rev.assign(shifted_month_id=pd.to_datetime(monthly_order_state_rev["month_id"].astype(str)) + pd.DateOffset(months=1))

In [57]:
# 9. shift month_ids by 12 months
monthly_order_state_rev = monthly_order_state_rev.assign(shifted_year_month_id=pd.to_datetime(monthly_order_state_rev["month_id"].astype(str)) + pd.DateOffset(months=12))

In [58]:
monthly_order_state_rev

,month_id,customer_state,revenue,shifted_month_id,shifted_year_month_id
0,2016-09,RR,72.89,2016-10-01,2017-09-01
1,2016-09,RS,59.50,2016-10-01,2017-09-01
2,2016-09,SP,134.97,2016-10-01,2017-09-01
3,2016-10,AL,82.49,2016-11-01,2017-10-01
4,2016-10,BA,894.05,2016-11-01,2017-10-01
...,...,...,...,...,...
554,2018-08,SC,32355.83,2018-09-01,2019-08-01
555,2018-08,SE,4558.04,2018-09-01,2019-08-01
556,2018-08,SP,396613.49,2018-09-01,2019-08-01
557,2018-08,TO,1925.97,2018-09-01,2019-08-01


In [62]:
# 10. convert month_id to datetime
monthly_order_state_rev["month_id"] = pd.to_datetime(monthly_order_state_rev["month_id"].astype(str))

In [70]:
# 11. perform the joins
monthly_state_growth = (monthly_order_state_rev[["month_id", "customer_state", "revenue"]]
                        .merge(monthly_order_state_rev[["shifted_month_id", "customer_state", "revenue"]],
                               how="left",
                               left_on=["month_id", "customer_state"],
                               right_on=["shifted_month_id", "customer_state"])
                        .merge(monthly_order_state_rev[["shifted_year_month_id", "customer_state", "revenue"]],
                               how="left",
                               left_on=["month_id", "customer_state"],
                               right_on=["shifted_year_month_id", "customer_state"])[["month_id",
                                                                                      "customer_state",
                                                                                      "revenue_x",
                                                                                      "revenue_y",
                                                                                      "revenue"]]
                        .rename({"revenue_x": "revenue",
                                 "revenue_y": "revenue_lm",
                                 "revenue": "revenue_ly"}, axis=1))

In [75]:
# 12. calculate growth
monthly_state_growth = (monthly_state_growth
                        .assign(mom_growth=monthly_state_growth["revenue"] / monthly_state_growth["revenue_lm"] - 1)
                        .assign(yoy_growth=monthly_state_growth["revenue"] / monthly_state_growth["revenue_ly"] - 1))

In [79]:
monthly_state_growth

,month_id,customer_state,revenue,revenue_lm,revenue_ly,mom_growth,yoy_growth
0,2016-09-01,RR,72.89,NaN,NaN,NaN,NaN
1,2016-09-01,RS,59.50,NaN,NaN,NaN,NaN
2,2016-09-01,SP,134.97,NaN,NaN,NaN,NaN
3,2016-10-01,AL,82.49,NaN,NaN,NaN,NaN
4,2016-10-01,BA,894.05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
554,2018-08-01,SC,32355.83,31203.58,22351.81,0.036927,0.447571
555,2018-08-01,SE,4558.04,4442.34,4821.69,0.026045,-0.054680
556,2018-08-01,SP,396613.49,326319.19,198753.97,0.215416,0.995500
557,2018-08-01,TO,1925.97,2613.67,2132.57,-0.263117,-0.096878


Validate

In [76]:
monthly_state_growth

,month_id,customer_state,revenue,revenue_lm,revenue_ly,mom_growth,yoy_growth
0,2016-09-01,RR,72.89,NaN,NaN,NaN,NaN
1,2016-09-01,RS,59.50,NaN,NaN,NaN,NaN
2,2016-09-01,SP,134.97,NaN,NaN,NaN,NaN
3,2016-10-01,AL,82.49,NaN,NaN,NaN,NaN
4,2016-10-01,BA,894.05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
554,2018-08-01,SC,32355.83,31203.58,22351.81,0.036927,0.447571
555,2018-08-01,SE,4558.04,4442.34,4821.69,0.026045,-0.054680
556,2018-08-01,SP,396613.49,326319.19,198753.97,0.215416,0.995500
557,2018-08-01,TO,1925.97,2613.67,2132.57,-0.263117,-0.096878


In [78]:
monthly_order_state_rev[(monthly_order_state_rev["month_id"] == "2018-08-01") & (monthly_order_state_rev["customer_state"] == "SP")]

,month_id,customer_state,revenue,shifted_month_id,shifted_year_month_id
556,2018-08-01,SP,396613.49,2018-09-01,2019-08-01
